## Path Settings

In [ ]:
import os
OPENPOSE_PATH="./openpose/"
HOME_PATH='./'

## Installing OpenPose

In [ ]:
#@title
!pip install ffmpeg-python
from os.path import exists, join, basename, splitext

# see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
# install new CMake becaue of CUDA10
!wget -q https://cmake.org/files/v3.17/cmake-3.17.2-Linux-x86_64.tar.gz
!tar xfz cmake-3.17.2-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

# install system dependencies
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
!rm -rf openpose
# clone openpose
!git clone -q --depth 1 $git_repo_url
# --recursive necessary in the line below, as otherwise you can (sometimes) get "lpthreads" errors in cmake ("undefined reference to `pthread_create'" etc). See, for example, https://github.com/facebookarchive/caffe2/issues/1234
!sed -i 's/execute_process(COMMAND git checkout --recursive master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
!cd openpose && git submodule update --init --recursive --remote

!wget "https://drive.usercontent.google.com/download?id=1QCSxJZpnWvM00hx49CJ2zky7PWGzpcEh&export=download&authuser=0&confirm=t&uuid=50355e54-0445-4875-9d0e-3866a1f6239d&at=APZUnTVA3sHsK5rk-u2O6phIms2L%3A1711987634035" -O models.zip

# !cp "/content/drive/MyDrive/VTO/openpose/models.zip" "/content/"
!unzip models.zip -d openpose

Selecting previously unselected package liblmdb0:amd64.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../00-liblmdb0_0.9.24-1build2_amd64.deb ...
Unpacking liblmdb0:amd64 (0.9.24-1build2) ...
Selecting previously unselected package libgflags2.2.
Preparing to unpack .../01-libgflags2.2_2.2.2-2_amd64.deb ...
Unpacking libgflags2.2 (2.2.2-2) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../02-libgflags-dev_2.2.2-2_amd64.deb ...
Unpacking libgflags-dev (2.2.2-2) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../03-libgoogle-glog0v5_0.5.0+really0.4.0-2_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.5.0+really0.4.0-2) ...
Selecting previously unselected package libunwind-dev:amd64.
Preparing to unpack .../04-libunwind-dev_1.3.2-2build2.1_amd64.deb ...
Unpacking libunwind-dev:amd64 (1.3.2-2build2.1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to un

In [ ]:
! sed -i 's/executeShInItsFolder "getModels.sh"/# executeShInItsFolder "getModels.sh"/g' ./openpose/scripts/ubuntu/install_openpose_JetsonTX2_JetPack3.1.sh
! sed -i 's/executeShInItsFolder "getModels.sh"/# executeShInItsFolder "getModels.sh"/g' ./openpose/scripts/ubuntu/install_openpose_JetsonTX2_JetPack3.3.sh
! sed -i 's/download_model("BODY_25"/# download_model("BODY_25"/g' ./openpose/CMakeLists.txt
! sed -i 's/78287B57CF85FA89C03F1393D368E5B7/# 78287B57CF85FA89C03F1393D368E5B7/g' ./openpose/CMakeLists.txt
! sed -i 's/download_model("body (COCO)"/# download_model("body (COCO)"/g' ./openpose/CMakeLists.txt
! sed -i 's/5156d31f670511fce9b4e28b403f2939/# 5156d31f670511fce9b4e28b403f2939/g' ./openpose/CMakeLists.txt
! sed -i 's/download_model("body (MPI)"/# download_model("body (MPI)"/g' ./openpose/CMakeLists.txt
! sed -i 's/2ca0990c7562bd7ae03f3f54afa96e00/# 2ca0990c7562bd7ae03f3f54afa96e00/g' ./openpose/CMakeLists.txt
! sed -i 's/download_model("face"/# download_model("face"/g' ./openpose/CMakeLists.txt
! sed -i 's/e747180d728fa4e4418c465828384333/# e747180d728fa4e4418c465828384333/g' ./openpose/CMakeLists.txt
! sed -i 's/download_model("hand"/# download_model("hand"/g' ./openpose/CMakeLists.txt
! sed -i 's/a82cfc3fea7c62f159e11bd3674c1531/# a82cfc3fea7c62f159e11bd3674c1531/g' ./openpose/CMakeLists.txt

## Make (takes \~ 10 minutes!)

In [ ]:
!cd openpose && rm -rf build || true && mkdir build

In [ ]:
cmake_file='/content/openpose/CMakeLists.txt'
!cd openpose && sed -i 's/-DBUILD_python=OFF/-DBUILD_python=ON/g' $cmake_file
!cd openpose && sed -i 's/-DBUILD_python_layer=OFF/-DBUILD_python_layer=ON/g' $cmake_file

!cd openpose && sed -i 's/option(BUILD_PYTHON "Build OpenPose python." OFF)/option(BUILD_PYTHON "OpenPose python." ON)\noption(BUILD_BIN_FOLDER "Copy 3rd-party DLL files." ON)/g' $cmake_file

In [ ]:
!cd openpose && cd build && cmake .. -DUSE_CUDNN=OFF -DGENERATE_PYTHON_BINDINGS:BOOL="1" -DPYTHON_LIBRARY='/usr/lib/x86_64-linux-gnu/libpython3.10.so' && make -j`nproc`

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc - works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ - works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- GCC detected, adding compile flags
-- GCC detected, adding compile flags
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE  
-- Found CUDA: /usr/local/cuda (found version "12.2") 
-- Building with CUDA.
-- CUDA detected: 12.2
-- Added CUDA NVCC flags for: sm_75
-- Found cuDNN

## Mount drive with dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Make the build executable

In [ ]:
!cd $OPENPOSE_PATH && chmod -R 755 './build/'

## Extract frames for all videos, pass them through openpose & save keypoints in drive

In [ ]:
os.mkdir("/content/output_jsons/")
os.mkdir("/content/frames")

In [ ]:
import os
import shutil
import subprocess
from tqdm import tqdm
path = "/content/drive/MyDrive/ataxia_dataset"
# Extract frames for all videos & save them
for video in os.listdir(path):
  if ".mp4" not in video:
    continue
  else:
    # Break the video into frames at 30 fps
    if os.path.exists(f"/content/frames/{video[:-4]}/"):
      continue
    os.mkdir(f"/content/frames/{video[:-4]}/")
    command = ["ffmpeg", "-i", os.path.join(path, video), "-vf", "fps=30", f"/content/frames/{video[:-4]}/output_%05d.jpg"]
    out = subprocess.run(command, stderr=subprocess.PIPE)

# Using these frames,
for video in tqdm(os.listdir(path)):
  if ".mp4" not in video:
    continue
  else:
    # Extract keypoints using openpose
    if os.path.exists(f"/content/output_jsons/{video[:-4]}/"):
      continue
    os.mkdir(f"/content/output_jsons/{video[:-4]}/")
    openposed = ["./build/examples/openpose/openpose.bin", "--image_dir", f"/content/frames/{video[:-4]}/", "--write_json", f"/content/output_jsons/{video[:-4]}/", "--display", "0", "--render_pose", "0", "--model_pose", "BODY_25"]
    openpoze = subprocess.run(openposed, stderr=subprocess.PIPE, cwd="/content/openpose")

100%|██████████| 153/153 [56:00<00:00, 21.97s/it]


'/content/drive/MyDrive/ataxia_dataset/keypoints.zip'

In [ ]:
# Zip the folder
!zip -r keypoints.zip /content/output_jsons/

Streaming output truncated to the last 5000 lines.
  adding: content/output_jsons/63/output_00170_keypoints.json (deflated 59%)
  adding: content/output_jsons/63/output_00055_keypoints.json (deflated 59%)
  adding: content/output_jsons/63/output_00172_keypoints.json (deflated 58%)
  adding: content/output_jsons/63/output_00073_keypoints.json (deflated 59%)
  adding: content/output_jsons/63/output_00162_keypoints.json (deflated 59%)
  adding: content/output_jsons/63/output_00062_keypoints.json (deflated 58%)
  adding: content/output_jsons/63/output_00078_keypoints.json (deflated 59%)
  adding: content/output_jsons/63/output_00084_keypoints.json (deflated 59%)
  adding: content/output_jsons/63/output_00094_keypoints.json (deflated 60%)
  adding: content/output_jsons/63/output_00056_keypoints.json (deflated 59%)
  adding: content/output_jsons/63/output_00006_keypoints.json (deflated 59%)
  adding: content/output_jsons/63/output_00104_keypoints.json (deflated 60%)
  adding: content/output_

In [ ]:
# Copy to drive
shutil.copy(f"/content/keypoints.zip", "/content/drive/MyDrive/ataxia_dataset")

'/content/drive/MyDrive/ataxia_dataset/keypoints.zip'

## Thank you!